In [1]:
localaddress = "http://YOUR_IP_ADDRESS:PORT"

In [2]:
import os

# 清除代理变量
os.environ.pop('http_proxy', None)
os.environ.pop('https_proxy', None)
os.environ.pop('ftp_proxy', None)
os.environ.pop('all_proxy', None)
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('FTP_PROXY', None)
os.environ.pop('ALL_PROXY', None)

# 验证代理变量是否已清除
print(os.environ.get('http_proxy'))
print(os.environ.get('https_proxy'))
print(os.environ.get('ftp_proxy'))
print(os.environ.get('all_proxy'))
print(os.environ.get('HTTP_PROXY'))
print(os.environ.get('HTTPS_PROXY'))
print(os.environ.get('FTP_PROXY'))
print(os.environ.get('ALL_PROXY'))

None
None
None
None
None
None
None
None


In [3]:
import selenium
import sys
import platform

print(f"Python版本: {sys.version}")
print(f"Selenium版本: {selenium.__version__}")
print(f"平台信息: {platform.platform()}")

# 检查关键模块
try:
    from selenium.webdriver.common.selenium_manager import SeleniumManager
    print("✓ Selenium Manager 模块可用")
except ImportError:
    print("✗ Selenium Manager 模块不可用")

# 检查 Edge 浏览器是否存在
import os
edge_path = r"/path/to/file"
msedge_path = r""

if os.path.exists(edge_path):
    print("✓ Edge 浏览器可执行文件存在")
else:
    print("✗ Edge 浏览器可执行文件不存在")

if os.path.exists(edge_path):
    print("✓ Edge driver 可执行文件存在")
else:
    print("✗ Edge driver 执行文件不存在")

Python版本: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]
Selenium版本: 4.31.1
平台信息: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
✓ Selenium Manager 模块可用
✓ Edge 浏览器可执行文件存在
✓ Edge driver 可执行文件存在


In [4]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import json

# 测试连接
try:
    response = requests.get(localaddress, timeout=5)
    print("✅ Windows EdgeDriver 服务正常")
except Exception as e:
    print(f"❌ 无法连接到 Windows 服务: {e}")

✅ Windows EdgeDriver 服务正常


In [5]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import requests
import sys

def init_driver():
    """初始化 Edge 浏览器驱动（远程 Windows EdgeDriver）"""
    # 0. 统一前置：检测 Windows EdgeDriver 是否可访问
    try:
        response = requests.get(localaddress, timeout=5)
        print("✅ Windows EdgeDriver 服务正常")
    except Exception as e:
        print(f"❌ 无法连接到 Windows 服务: {e}")
        raise RuntimeError("网络不可达，终止后续操作")

    # 1. 配置选项
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--start-maximized')
    # options.add_argument('--headless')

    # 2. 建立远程会话
    try:
        driver = webdriver.Remote(command_executor=localaddress, options=options)
        print("✅ 浏览器驱动初始化成功")
        return driver
    except Exception as e:
        print(f"❌ 驱动初始化失败: {e}")
        return None


def smoke_test(driver):
    """简单冒烟测试：打开 Bing 并拿标题"""
    try:
        driver.get("https://example.com")
        title = driver.title
        print(f"✅ 测试通过，页面标题：{title}")
        return True
    except Exception as e:
        print(f"❌ 测试失败：{e}")
        return False


# ---------------- 主流程 ----------------
driver = init_driver()
if driver is None:
    sys.exit(1)

try:
    if not smoke_test(driver):
        # 测试失败 → 不关闭浏览器，方便人工排查
        sys.exit(2)
finally:
    if driver is not None:
        driver.quit()          # 测试成功才关闭
        print("🚪 浏览器已关闭")

✅ Windows EdgeDriver 服务正常


✅ 浏览器驱动初始化成功


✅ 测试通过，页面标题：搜索 - Microsoft 必应


🚪 浏览器已关闭


In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pickle
import os
import time
import csv

def load_cookies(driver, cookie_file='cookies/tools_zhihu_cookies.pkl'):
    """加载Cookie到浏览器会话"""
    if not os.path.exists(cookie_file):
        print(f"❌ Cookie文件不存在: {cookie_file}")
        return False
    
    try:
        with open(cookie_file, 'rb') as file:
            cookies = pickle.load(file)
        
        print(f"📁 加载了 {len(cookies)} 个Cookie")
        
        # 先访问域名根路径
        driver.get("https://example.com")  # 根据实际情况调整域名
        time.sleep(2)
        
        # 清除可能存在的旧Cookie
        driver.delete_all_cookies()
        
        # 添加所有Cookie
        cookies_loaded = 0
        for cookie in cookies:
            try:
                driver.add_cookie(cookie)
                cookies_loaded += 1
            except Exception as e:
                print(f"⚠️ 跳过Cookie {cookie.get('name', 'unknown')}: {e}")
        
        print(f"✅ 成功加载 {cookies_loaded} 个Cookie")
        return True
        
    except Exception as e:
        print(f"❌ 加载Cookie失败: {e}")
        return False

def extract_search_index(driver, url, timeout=30):
    """
    从网页中提取搜索指数
    
    参数:
        driver: 浏览器驱动
        url: 要提取数据的网页URL
        timeout: 超时时间
        
    返回:
        搜索指数数值（整数）或 None（如果提取失败）
    """
    try:
        print(f"🌐 访问URL: {url}")
        driver.get(url)
        
        # 等待页面加载
        time.sleep(5)
        
        # 检查是否成功访问（没有重定向到登录页面）
        current_url = driver.current_url
        if "login" in current_url.lower():
            print("❌ 被重定向到登录页面，Cookie可能已失效")
            return None
        
        # 获取页面源码
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # 方法1: 使用BeautifulSoup查找包含"搜索指数"的span和对应的数值span
        search_index_value = extract_with_bs4(soup)
        
        if search_index_value is not None:
            print(f"✅ 提取到搜索指数: {search_index_value}")
            return search_index_value
        
        # 方法2: 如果BeautifulSoup失败，尝试使用Selenium直接查找
        search_index_value = extract_with_selenium(driver)
        
        if search_index_value is not None:
            print(f"✅ 提取到搜索指数: {search_index_value}")
            return search_index_value
        
        print("❌ 未找到搜索指数数据")
        return None
        
    except Exception as e:
        print(f"❌ 提取数据时出错: {e}")
        return None

def extract_with_bs4(soup):
    """使用BeautifulSoup提取搜索指数"""
    try:
        # 查找包含"搜索指数"文本的span元素
        search_index_span = soup.find('span', string='搜索指数')
        
        if search_index_span:
            # 在找到的span附近查找数值span
            # 根据您提供的HTML结构，数值在具有特定class的span中
            num_weight_span = search_index_span.find_next('span', class_=lambda x: x and 'numWeight' in x if x else False)
            
            if num_weight_span:
                value_text = num_weight_span.get_text(strip=True)
                return clean_numeric_value(value_text)
            
            # 如果没找到特定class的span，尝试查找后面的span兄弟元素
            next_span = search_index_span.find_next_sibling('span')
            if next_span:
                value_text = next_span.get_text(strip=True)
                return clean_numeric_value(value_text)
        
        return None
    except Exception as e:
        print(f"❌ BeautifulSoup提取失败: {e}")
        return None

def extract_with_selenium(driver):
    """使用Selenium直接提取搜索指数"""
    try:
        # 等待包含"搜索指数"的元素出现
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(), '搜索指数')]"))
        )
        
        # 查找数值元素 - 尝试多种可能的定位方式
        selectors = [
            "//span[contains(@class, 'numWeight')]",
            "//span[contains(text(), '搜索指数')]/following-sibling::span",
            "//span[contains(text(), '搜索指数')]/../span[contains(@class, 'numWeight')]"
        ]
        
        for selector in selectors:
            try:
                value_element = driver.find_element(By.XPATH, selector)
                value_text = value_element.text.strip()
                if value_text and value_text != "搜索指数":
                    return clean_numeric_value(value_text)
            except:
                continue
        
        return None
    except Exception as e:
        print(f"❌ Selenium提取失败: {e}")
        return None

def clean_numeric_value(value_text):
    """清理数值文本，转换为整数"""
    if not value_text:
        return None
    
    # 移除逗号等非数字字符（保留数字和点）
    clean_text = ''.join(c for c in value_text if c.isdigit() or c == '.')
    
    try:
        # 转换为整数
        return int(float(clean_text)) if clean_text else None
    except ValueError:
        return value_text  # 如果转换失败，返回原始文本

def process_multiple_urls(urls, output_csv='toolsZhihu.csv', cookie_file='cookies/tools_zhihu_cookies.pkl'):
    """
    处理多个URL，提取搜索指数并保存到CSV
    
    参数:
        urls: URL列表
        output_csv: 输出CSV文件名
        cookie_file: Cookie文件路径
    """
    print("🚀 开始处理多个网页的搜索指数数据...")
    
    # 初始化浏览器
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--start-maximized')
    
    driver = None
    results = []
    
    try:
        # 连接到远程EdgeDriver
        driver = webdriver.Remote(
            command_executor=localaddress,
            options=options
        )
        print("✅ 浏览器初始化成功")
        
        # 加载Cookie
        if not load_cookies(driver, cookie_file):
            print("❌ Cookie加载失败，退出")
            return False
        
        # 处理每个URL
        for i, url in enumerate(urls, 1):
            print(f"\n📊 处理第 {i}/{len(urls)} 个URL")
            
            search_index = extract_search_index(driver, url)
            
            results.append({
                'URL': url,
                '搜索指数': search_index if search_index is not None else '提取失败',
                '处理时间': time.strftime('%Y-%m-%d %H:%M:%S')
            })
            
            # 短暂休息，避免请求过快
            time.sleep(2)
        
        # 保存结果到CSV
        if results:
            save_results_to_csv(results, output_csv)
            print(f"🎉 处理完成！共处理 {len(results)} 个网页")
            return True
        else:
            print("❌ 未提取到任何数据")
            return False
            
    except Exception as e:
        print(f"❌ 处理过程中出错: {e}")
        return False
    finally:
        if driver:
            driver.quit()
            print("🔚 浏览器已关闭")

def save_results_to_csv(results, output_file):
    """将结果保存到CSV文件"""
    try:
        # 使用csv模块确保正确处理中文字符
        with open(output_file, 'w', newline='', encoding='utf-8-sig') as f:
            if results:
                fieldnames = results[0].keys()
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                for result in results:
                    writer.writerow(result)
        
        print(f"💾 数据已保存到: {output_file}")
        
        # 显示数据预览
        df = pd.DataFrame(results)
        print("\n📋 数据预览:")
        print(df.to_string(index=False))
        
        # 统计成功提取的数据
        success_count = sum(1 for r in results if r['搜索指数'] != '提取失败')
        print(f"\n📈 成功提取 {success_count}/{len(results)} 个网页的数据")
        
    except Exception as e:
        print(f"❌ 保存CSV文件失败: {e}")

def append_to_existing_csv(new_results, existing_file='搜索指数数据.csv'):
    """向已存在的CSV文件追加新数据"""
    try:
        # 检查文件是否存在
        file_exists = os.path.exists(existing_file)
        
        with open(existing_file, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = new_results[0].keys() if new_results else ['URL', '搜索指数', '处理时间']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
            
            for result in new_results:
                writer.writerow(result)
        
        mode = "追加到" if file_exists else "创建"
        print(f"✅ 已{mode}文件: {existing_file}")
        
    except Exception as e:
        print(f"❌ 追加数据失败: {e}")

# 在Jupyter中使用的便捷函数
def jupyter_process_urls(urls, cookie_file='cookies/tools_zhihu_cookies.pkl'):
    """
    Jupyter专用函数
    """
    results = process_multiple_urls(urls, cookie_file=cookie_file)
    if results:
        df = pd.DataFrame(results)
        display(df)  # 在Jupyter中显示表格
        return df
    else:
        return None

# 使用示例
if __name__ == "__main__":
    # 定义要处理的URL列表
    urls = [
        #股票
        "https://example.com",
        #经济
        "https://example.com",
    ]
    
    # 处理URLs并保存结果
    process_multiple_urls(urls, "toolsZhihu.csv")

🚀 开始处理多个网页的搜索指数数据...


✅ 浏览器初始化成功
📁 加载了 11 个Cookie


⚠️ 跳过Cookie ll_b: Message: invalid cookie domain: Cookie 'domain' mismatch
  (Session info: MicrosoftEdge=143.0.3650.96)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff63eb89435
	0x7ff63eaf9aa4
	0x7ff63eee2d12
	0x7ff63e99e469
	0x7ff63e9723ea
	0x7ff63e94a2e5
	0x7ff63e98c8de
	0x7ff63e94982a
	0x7ff63e948b33
	0x7ff63e949653
	0x7ff63ea31d64
	0x7ff63ea40b1c
	0x7ff63ea3a6ff
	0x7ff63ec1a7f7
	0x7ff63eb05306
	0x7ff63eaff714
	0x7ff63eaff859
	0x7ff63eaf391d
	0x7ffafbe47374
	0x7ffafddbcc91



⚠️ 跳过Cookie BEC: Message: invalid cookie domain: Cookie 'domain' mismatch
  (Session info: MicrosoftEdge=143.0.3650.96)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff63eb89435
	0x7ff63eaf9aa4
	0x7ff63eee2d12
	0x7ff63e99e469
	0x7ff63e9723ea
	0x7ff63e94a2e5
	0x7ff63e98c8de
	0x7ff63e94982a
	0x7ff63e948b33
	0x7ff63e949653
	0x7ff63ea31d64
	0x7ff63ea40b1c
	0x7ff63ea3a6ff
	0x7ff63ec1a7f7
	0x7ff63eb05306
	0x7ff63eaff714
	0x7ff63eaff859
	0x7ff63eaf391d
	0x7ffafbe47374
	0x7ffafddbcc91

✅ 成功加载 9 个Cookie

📊 处理第 1/2 个URL
🌐 访问URL: https://tools.zhihu.com/questionselection/buzzwordDetail?word=-3278820883975706256


✅ 提取到搜索指数: 1316



📊 处理第 2/2 个URL
🌐 访问URL: https://tools.zhihu.com/questionselection/buzzwordDetail?word=7363040180555765370


❌ 未找到搜索指数数据


💾 数据已保存到: toolsZhihu.csv

📋 数据预览:
                                                                               URL 搜索指数                处理时间
https://tools.zhihu.com/questionselection/buzzwordDetail?word=-3278820883975706256 1316 2026-01-07 16:37:02
 https://tools.zhihu.com/questionselection/buzzwordDetail?word=7363040180555765370 提取失败 2026-01-07 16:37:10

📈 成功提取 1/2 个网页的数据
🎉 处理完成！共处理 2 个网页


🔚 浏览器已关闭


In [7]:
from pathlib import Path

# 获取当前目录
current_dir = Path('.')

# 查找并删除所有 .html 文件
html_files = list(current_dir.glob('*.html'))
for file in html_files:
    file.unlink()  # 删除文件
    print(f"已删除: {file.name}")

print(f"\n总共删除了 {len(html_files)} 个 .html 文件")


总共删除了 0 个 .html 文件
